The aim of this notebook is to demonstrate how to implement logistic regression with stochastic gradient descent. 

In this project, we are trying to do gender voice recognition with logistic regression. The data is obtained from kaggle, where each voice signal has already been analyzed and the data about the voice samples are recorded numerically. 

# Prepare notebook and import data

In [7]:
import pandas as pd
import numpy as np

In [8]:
# import the data and see what is in there (first ten lines)

df_voice=pd.read_csv("voice.csv")
df_voice.shape

(3168, 21)

Taking a look at the dataframe itself:

In [9]:
df_voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


All the variables are represented below:

In [10]:
list(df_voice.columns)

['meanfreq',
 'sd',
 'median',
 'Q25',
 'Q75',
 'IQR',
 'skew',
 'kurt',
 'sp.ent',
 'sfm',
 'mode',
 'centroid',
 'meanfun',
 'minfun',
 'maxfun',
 'meandom',
 'mindom',
 'maxdom',
 'dfrange',
 'modindx',
 'label']

Since label is now a categorical data, split into males and females, it would be beneficial to convert them to 0 and 1, where 0 represent females and 1 represent males

In [11]:
gender_dict = {"male":1, "female":0}
df_voice["gender"] = df_voice["label"].map(gender_dict);
df_voice = df_voice.drop("label",1)

In [12]:
df_voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,gender
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,1
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,1
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,1
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,1


This is the data that we have, and from the data, we observe several problems:

1. The data is listed in such a way that the male voices comes first and female voices comes later, therefore we need to shuffle the rows to ensure that both male and female voices are taken into account of the learning process.
2. Males and females are label, and we need to change them into 0 and 1 so that we can fit the data into the logistic regression algorithm
3. We need a set of testing data to test the algorithm. 

In [13]:
df_voice = df_voice.iloc[np.random.permutation(len(df_voice))]

It is also beneficial to split the data into training and testing data, so that we can verify whether logistic regression can accurately predict the gender of a voice. 

In [14]:
df_test = pd.DataFrame()
df_train = pd.DataFrame()
df_train = df_voice[:2100]
df_test = df_voice[2100:]

In [15]:
df_train.shape

(2100, 21)

In [16]:
# Now we split the testing data to a column vector of gender and
y_train = df_train["gender"].as_matrix()
X_train = df_train.drop("gender",1).as_matrix()

In [17]:
y_train = y_train.transpose()
X_train = X_train.transpose()

In [18]:
X_train.shape

(20L, 2100L)

In [42]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [49]:
def stochGradDescent(outputVector,inputMatrix,learningSpeed):
    #outputVector should be a binary column vector of classifications
    a = learningSpeed
    y = outputVector
    X = inputMatrix
    numberOfTrials = y.shape[0]
    numberOfEstimators = X.shape[0]
    t = np.zeros(numberOfEstimators) #estimator vector (_t_heta)
    
    for i in range( numberOfTrials ):
        grad = (y[i] - sigmoid(np.dot(np.transpose(t),X[:,i])))
        t = t + a * grad * X[:,i]
        print t[1]
        
    return t

In [54]:
theta = stochGradDescent(y_train,X_train,0.0001)
theta

-4.45752609655e-06
-7.71375999175e-06
-1.12837541793e-05
-1.27784666158e-05
-1.52025286035e-05
-1.67409471276e-05
-1.82610721273e-05
-1.53518096921e-05
-1.22175388249e-05
-8.99995265103e-06
-5.29705979437e-06
-2.3748681596e-06
6.35445303082e-07
-3.30287227844e-06
7.21406552758e-07
4.67440991239e-06
2.49072429249e-06
1.08352201439e-07
-2.04700871792e-06
-4.58041730385e-06
-8.36012998518e-06
-1.0072068437e-05
-7.07740736937e-06
-4.29807172355e-06
-1.20446620951e-06
1.87664176784e-06
4.65691928136e-06
2.69843507662e-06
5.13191485826e-06
8.75374116507e-06
5.06665362202e-06
1.9412172991e-06
4.96008045821e-06
3.35089373518e-06
6.61928698538e-07
-2.66604726366e-06
1.5270634432e-07
3.64455844331e-06
6.56787733163e-06
8.55861409278e-06
4.86061687402e-06
8.98595732198e-06
1.19722911945e-05
1.00065258292e-05
1.28785739532e-05
1.6532784569e-05
1.94736159798e-05
1.77460448654e-05
1.5469033154e-05
1.36623244237e-05
1.72231231936e-05
2.00041937206e-05
2.35575377223e-05
2.62358151375e-05
2.41136147786

array([ -2.11972348e-05,   1.02848949e-03,   1.64934529e-05,
        -1.67115290e-03,   1.30942851e-03,   2.98058141e-03,
        -1.35751358e-02,  -9.02442722e-03,   6.60710226e-03,
         7.92618160e-03,  -1.30660639e-04,  -2.11972348e-05,
        -2.02676911e-03,  -2.65652735e-05,   7.45530413e-04,
        -3.49350100e-03,  -1.04057071e-03,  -1.85808928e-02,
        -1.75403221e-02,   9.18978570e-04])

In [33]:
def predict(inputVector,estimatorVector):
    t = estimatorVector
    x = inputVector
    confidence = sigmoid(np.dot(np.transpose(t) , x))
    if confidence > 0.5:
        return 1
    else:
        return 0

In [38]:
predict(X_train[:,1223],theta)

1

In [39]:
y_train[1223]

0

3.7200759760208356e-44